In [1]:
import requests as req
import json as js
from dtos.dtos import *
from dtos.dtos_two_wheels import *

In [2]:
baseUrl:str = "http://localhost:3010/maas/twowheels"
baseHeader = {"user" : "{'_id' : '6138aa9c8b39445f30a246a1', 'client_id' : '60f1b8e0203c9b20acb9ad26'}", 
              "Content-Type" : "application/json"}
main_date = "2024-05-28T15:00:36.459Z"
provider_id = "0b13f2d6-d5cb-48aa-aeed-ed3a5a7f1da6"

<h1> Get all providers </h1>

In [5]:
response = req.request("GET", 
                       baseUrl + "/providers", 
                       headers=baseHeader)
result : TwoWheelsAllProvidersDTO = TwoWheelsAllProvidersDTOSchema().loads(response.text)
result

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

<Response [200]>

<h1> Get configuration </h1>

In [7]:
response = req.request("GET", 
                       baseUrl + "/moto-rental/configuration", 
                       headers=baseHeader,
                       params={"providerCode":"cooltra"})
result : CooltraConfigurationResponseDTO = CooltraConfigurationResponseDTOSchema().loads(response.text)
all_ids = [system.id for system in result.systems]

In [8]:
all_ids

['rome',
 'paris',
 'madrid',
 'berlin',
 'zurich',
 'valencia',
 'seville',
 'turin',
 'lisbon',
 'barcelona',
 'milan']

<h1> Get vehicles in specific zone </h1>

In [9]:
response = req.request("GET", 
                       baseUrl + "/moto-rental/vehicles", 
                       headers=baseHeader, 
                       params={"providerCode":"cooltra", "systemId" : "barcelona"})
vehicles:List[CooltraMotoRentalVehicleDTO] = CooltraMotoRentalVehicleDTOSchema().loads(response.text, many=True)
first_vehicle_id = vehicles[8].id

<h1> Get specific vehicle </h1>

In [10]:
response = req.request("GET", 
                       baseUrl + f"/moto-rental/vehicles/{first_vehicle_id}", 
                       headers=baseHeader, 
                       params={"providerCode":"cooltra", "systemId" : "rome"})
result : CooltraMotoRentalVehicleDTO = CooltraMotoRentalVehicleDTOSchema().loads(response.text)

In [11]:
result

CooltraMotoRentalVehicleDTO(id=15748800875, license_plate='FAKE-1543829080-0875', position=[2.1704042403691215, 41.403388890773556], range=50, battery_percentage=44, model_id=2)

<h1> Reserve vehicle </h1>

In [72]:
reservationBody = CooltraReservationRequestDTO(
    vehicle_id = first_vehicle_id,
    provider_code="cooltra"
)

response = req.request("POST", 
                       baseUrl + f"/moto-rental/reserve", 
                       headers=baseHeader, 
                       data=CooltraReservationRequestDTOSchema().dumps(reservationBody))

result : CooltraReservationInfoDTOSchema = CooltraReservationInfoDTOSchema().loads(response.text)

In [73]:
response.text

'{"id":"5242784563847710390","system_id":"barcelona","created_at":"2024-06-20T08:39:23.9517986Z","expires_at":"2024-06-20T08:54:23.9517986Z","expires_in":900,"vehicle":{"id":"14856004550","license_plate":"FAKE-1543829080-04550","position":[2.173436423357217,41.40038157906233],"range":50,"battery_percentage":57,"model_id":"2"}}'

<h1> Cancel Reservation </h1>

In [74]:
cancellationBody = CooltraReservationCancellationRequestDTO(
    provider_code="cooltra"
)

response = req.request("DELETE", 
                       baseUrl + f"/moto-rental/cancel", 
                       headers=baseHeader, 
                       data=CooltraReservationRequestDTOSchema().dumps(reservationBody))

response.status_code

200

In [75]:
response.status_code

200

<h1> Check in to vehicle </h1>

In [24]:
reservationBody = CooltraCheckInRequestDTO(
    provider_code = "cooltra",
    vehicle_id = first_vehicle_id
)

response = req.request("POST", 
                       baseUrl + f"/moto-rental/checkin", 
                       headers=baseHeader, 
                       data=CooltraCheckInRequestDTOSchema().dumps(reservationBody))

result : CooltraCheckInInfoDTO = CooltraCheckInInfoDTOSchema().loads(response.text)

ValidationError: {'reservation_id': ['Field may not be null.']}

In [25]:
response.text

'{"id":"7140111388853094489","system_id":"barcelona","reservation_id":null,"started_at":"2024-06-20T10:28:49.824725Z","duration":0,"state":"active","vehicle":{"id":"15748800875","license_plate":"FAKE-1543829080-0875","position":[2.1704042403691215,41.403388890773556],"range":50,"battery_percentage":44,"model_id":"2"}}'

<h1> Get current activity </h1>

In [21]:
response = req.request("GET", 
                       baseUrl + f"/moto-rental/current/activity",
                       params={"providerCode" : "cooltra"},
                       headers=baseHeader)

result : CooltraCurrentActivityResponseDTO = CooltraCurrentActivityResponseDTOSchema().loads(response.text)
response.text

'{"reservation":null,"rental":null}'

In [22]:
result

CooltraCurrentActivityResponseDTO(reservation=None, rental=None)

<h1> Check out to vehicle </h1>

In [26]:
reservationBody = CooltraCheckoutRequestDTO(
    dismiss=[],
    provider_code="cooltra"
)

response = req.request("POST", 
                       baseUrl + f"/moto-rental/checkout", 
                       headers=baseHeader, 
                       data=CooltraCheckoutRequestDTOSchema().dumps(reservationBody))

result : CooltraCheckInInfoDTO = CooltraCheckInInfoDTOSchema().loads(response.text)

ValidationError: {'id': ['Missing data for required field.'], 'system_id': ['Missing data for required field.'], 'reservation_id': ['Missing data for required field.'], 'started_at': ['Missing data for required field.'], 'duration': ['Missing data for required field.'], 'state': ['Missing data for required field.'], 'vehicle': ['Missing data for required field.'], 'errors': ['Unknown field.'], 'code': ['Unknown field.']}

In [27]:
response.text

'{"errors":["Object reference not set to an instance of an object."],"code":"generic_error"}'